In [3]:
import pickle
# Load Picked file
with open('/l/users/shikhar.srivastava/workspace/hover_net/trail.pkl', "rb") as pkl_file:
    raw_data = pickle.load(pkl_file)

In [10]:
len(raw_data['imgs'])

336

In [12]:
raw_data.keys()

dict_keys(['imgs', 'true_np', 'true_hv', 'prob_np', 'pred_hv', 'true_tp', 'pred_tp'])

In [14]:
a = dict()

In [16]:
nr_types = 6

In [40]:
len(raw_data["true_np"])

336

In [47]:
raw_data['true_np'][0].shape

(164, 164)

In [43]:
import numpy as np
np.size(raw_data["true_np"][0])

26896

## Second Order Metrics

In [2]:
import pandas as pd
import numpy as np
import json
bucket_type = 'top5'

type_list_path = f'/l/users/shikhar.srivastava/data/pannuke/{bucket_type}/selected_types.csv'
type_list = pd.read_csv(type_list_path)['0']
metrics = pd.DataFrame(columns=['source_type','target_type', 'acc', 'dice', 'loss'])
for source_type in type_list:
    for target_type in type_list:
        logs_path = f'/l/users/shikhar.srivastava/workspace/hover_net/logs/second_order/{bucket_type}/ckpts/{source_type}-{target_type}/stats.json'
        with open(logs_path) as f:
            logs = json.load(f)['50']
        metrics = metrics.append({'source_type': source_type, 'target_type': target_type, 'acc': logs['valid-np_acc'], 'dice': logs['valid-np_dice'], 'loss': logs['train-overall_loss']}, ignore_index=True)


In [38]:
acc_pivot = metrics.pivot_table(index='source_type', columns='target_type', values='acc')

In [43]:
acc_pivot

target_type,Adrenal_gland,Bile-duct,Breast,Colon,Esophagus
source_type,,,,,
Adrenal_gland,0.975781,0.970176,0.953206,0.944140,0.959045
Bile-duct,0.974200,0.971785,0.952998,0.942867,0.959940
Breast,0.973522,0.969827,0.957857,0.946156,0.959767
Colon,0.971806,0.969518,0.954024,0.951965,0.957458
Esophagus,0.974043,0.969925,0.952683,0.942557,0.962185


In [58]:
# Github path


source_type
Adrenal_gland    0.953206
Bile-duct        0.952998
Breast           0.957857
Colon            0.954024
Esophagus        0.952683
Name: Breast, dtype: float64

In [61]:
W = np.zeros(shape = (len(type_list), len(type_list) - 1, len(type_list) - 1))
labels = []
for t, target_type in enumerate(type_list):
    sources = set(type_list)
    sources.remove(target_type)

    print('target_type: ', target_type)
    n_sources = len(sources) # All but the target, can be source types
    W_t = np.zeros((n_sources, n_sources))
    labels_t = list(sources)

    for i, source_i in enumerate(sources):
        for j, source_j in enumerate(sources):
            print(source_i, source_j)
            W_t[i, j] = acc_pivot.loc[source_i, target_type] - acc_pivot.loc[source_j, target_type]
        
    W[t, :, :] = W_t
    labels.append(labels_t)
        

target_type:  Breast
Colon Colon
0.0
Colon Esophagus
0.0013410340712727287
Colon Bile-duct
0.0010261085025919936
Colon Adrenal_gland
0.0008180761522336155
target_type:  Colon
Esophagus Esophagus
0.0
Esophagus Bile-duct
-0.0003102780374493186
Esophagus Breast
-0.0035995129387269564
Esophagus Adrenal_gland
-0.0015829270063171252
target_type:  Adrenal_gland
Colon Colon
0.0
Colon Esophagus
-0.0022371223548908192
Colon Bile-duct
-0.0023941425172091524
Colon Breast
-0.001716377545961878
target_type:  Esophagus
Colon Colon
0.0
Colon Bile-duct
-0.0024821136440894565
Colon Breast
-0.0023093803999887363
Colon Adrenal_gland
-0.0015871319155265073
target_type:  Bile-duct
Colon Colon
0.0
Colon Esophagus
-0.0004077655378034173
Colon Breast
-0.0003095034489107862
Colon Adrenal_gland
-0.0006587322242995741


In [21]:
metrics.pivot_table(index='source_type', columns='target_type', values='acc')

target_type,Adrenal_gland,Bile-duct,Breast,Colon,Esophagus
source_type,,,,,
Adrenal_gland,0.975781,0.970176,0.953206,0.944140,0.959045
Bile-duct,0.974200,0.971785,0.952998,0.942867,0.959940
Breast,0.973522,0.969827,0.957857,0.946156,0.959767
Colon,0.971806,0.969518,0.954024,0.951965,0.957458
Esophagus,0.974043,0.969925,0.952683,0.942557,0.962185


In [22]:
metrics.pivot_table(index='source_type', columns='target_type', values='dice')

target_type,Adrenal_gland,Bile-duct,Breast,Colon,Esophagus
source_type,,,,,
Adrenal_gland,0.901478,0.889520,0.864667,0.834428,0.891240
Bile-duct,0.895150,0.896330,0.863819,0.833163,0.893341
Breast,0.892318,0.888322,0.877305,0.840373,0.891873
Colon,0.884580,0.887132,0.866169,0.856614,0.887477
Esophagus,0.893824,0.888964,0.861948,0.830506,0.899792


In [23]:
metrics.pivot_table(index='source_type', columns='target_type', values='loss')

target_type,Adrenal_gland,Bile-duct,Breast,Colon,Esophagus
source_type,,,,,
Adrenal_gland,2.889199,3.414583,3.601528,3.578019,3.091034
Bile-duct,3.098608,3.292096,3.492840,3.563899,3.076532
Breast,3.050590,3.442894,3.142640,3.440628,3.110794
Colon,3.233108,3.519279,3.447159,3.018823,3.171385
Esophagus,3.117095,3.447486,3.626762,3.613839,2.943885
